In [1]:
import warnings 
warnings.simplefilter('ignore')

import os
import gc
import csv
import json
import copy
import random

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F

%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

from transformers import T5ForConditionalGeneration, BartForConditionalGeneration,AutoModelForSeq2SeqLM

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup, AutoConfig
import transformers

from evaluate import CiderD

In [2]:
def split_data(data_path='./data/train.csv'):
    if not os.path.exists('./data/test.csv'):
        df = pd.read_csv(data_path)
        train_df = df[:18000].reset_index(drop=True)
        test_df = df[18001:].reset_index(drop=True)
        
        train_df.to_csv('./data/train_df.csv', header=None, index=False)
        test_df.to_csv('./data/test_df.csv', header=None, index=False)
split_data()

In [3]:
class Config:
    def __init__(self):
        super(Config, self).__init__()
        
        self.SEED = 42
        self.tokenizer_path = 'bert_model/nezha-cn-base'
        self.MODEL_PATH = './pretrain/pretrained_0.15_dual/checkpoint-20000'
        
        # data
        self.max_length = 256 #句子最大长度
        self.batch_size = 32
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.full_finetuning = True
        self.lr = 3e-5
        self.optimizer = 'AdamW'
        self.n_warmup = 0
        self.save_best_only = True
        
        self.multi_gpu = False
        self.attack = 'pgd'

        #self.ema = True
        self.flooding = False
        self.loss_func = 'ce' # ce dice fl
        self.epochs = 20
config = Config()

In [4]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


np.random.seed(config.SEED)
seed_everything(seed=config.SEED)

In [5]:
class GAIIC_Dataset(Dataset):
    def __init__(self, data_file, input_len, output_len, sos_id=1, eos_id=2, pad_id=0):
        super(GAIIC_Dataset, self).__init__()
        
        with open(data_file, 'r') as fp:
            reader = csv.reader(fp)
            self.samples = [row for row in reader]#[:100]
            
        self.input_len   = input_len
        self.output_len  = output_len
        self.sos_id      = sos_id
        self.eos_id      = eos_id
        self.pad_id      = pad_id
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        source = [int(x) for x in self.samples[index][1].split()]
        
        if len(source) < self.input_len:
            source.extend([self.pad_id] * (self.input_len - len(source)))
        
        # for test
        if len(self.samples[index]) < 3:
            return np.array(source)[:self.input_len]
        
        target = [self.sos_id] + [int(x) for x in self.samples[index][2].split()] + [self.eos_id]
        if len(target) < self.output_len:
            target.extend([self.pad_id] * (self.output_len - len(target)))
            
        return  np.array(source)[:self.input_len], np.array(target)[:self.output_len]

In [6]:
class T5_Model(nn.Module):
    def __init__(self):
        super(T5_Model, self).__init__()

        self.model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base')
        
        self.PAD_ID = 0
        self.SOS_ID = 1
        self.EOS_ID = 2
        
        self.register_buffer('true_rep', torch.tensor(1.0))
        self.register_buffer('false_rep', torch.tensor(0.0))

    def forward(self, source_tokens, target_tokens=None, max_length=150, generate=False):
        
        if target_tokens is not None:
            feed_tensor = target_tokens[:, :-1]
            target_tensor = target_tokens.detach().clone()[:, 1:]
            input_pad_mask = torch.where(source_tokens!=self.PAD_ID, self.true_rep, self.false_rep)
            output_pad_mask = torch.where(target_tensor!=self.PAD_ID, self.true_rep, self.false_rep)
            target_tensor[target_tensor==self.PAD_ID]=-100
        
        if not generate:
            outputs = self.model(
                            input_ids = source_tokens,
                            attention_mask=input_pad_mask,
                            decoder_input_ids=feed_tensor,
                            use_cache=None,
                            return_dict=False
                )
            return outputs[0], target_tensor
        else:
            outputs = self.model.generate(
                            input_ids = source_tokens,
                            max_length=max_length,
                )
            
            return outputs

In [7]:
def array2str(arr):
    out = ''
    for i in range(len(arr)):
        if arr[i]==0 or arr[i]==2:
            break
        if arr[i]==1:
            continue
        out += str(int(arr[i])) + ' '
    if len(out.strip())==0:
        out = '0'
        
    return out.strip()

In [8]:
def val_fn(model, valid_dataloader):
    val_loss = 0

    model.eval()
    res, gts = [], {}
    tot = 0
    device = 'cuda'
    for batch_idx, (input_ids, target_ids) in enumerate(tqdm(valid_dataloader)):
        
        input_ids  = input_ids.to(device)
        target_ids = target_ids.cpu().numpy()
        
        pred = model(source_tokens=input_ids, 
                     max_length=150,
                     generate=True).cpu().numpy()
        
        for i in range(pred.shape[0]):
            res.append({'image_id':tot, 'caption': [array2str(pred[i][2:])]})
            gts[tot] = [array2str(target_ids[i][1:])]
            tot += 1
#             if i == 1:
#                 print('pred', pred[i])
#                 print('array2str(pred[i])', array2str(pred[i][2:]))
#                 print('array2str(target_ids[i])', array2str(target_ids[i]))
            
    CiderD_scorer = CiderD(df='corpus', sigma=15)
    cider_score,cider_scores = CiderD_scorer.compute_score(gts, res)
    print("cider_score:", cider_score, cider_scores)
    
    model.train()

    return cider_score, cider_scores

In [9]:
def test(model, test_dataloader):
    val_loss = 0

    model.eval()
    res = []
    tot = 0
    device = 'cuda'
    for batch_idx, input_ids in enumerate(tqdm(test_dataloader)):
        
        input_ids  = input_ids.to(device)
        
        pred = model(source_tokens=input_ids, 
                     max_length=150,
                     generate=True).cpu().numpy()
        
        for i in range(pred.shape[0]):
            res.append({'image_id':tot, 'caption': [array2str(pred[i][2:])]})
            tot += 1

#             if i == 1:
#                 print('pred', pred[i])
#                 print('array2str(pred[i])', array2str(pred[i][2:]))
#                 print('array2str(target_ids[i])', array2str(target_ids[i]))
    return res

In [10]:
def train_fn(model, train_dataloader, criterion, optimizer, scheduler=None, epoch=0):
    
    train_loss = 0
    model.train()
    device = 'cuda'

    for batch_idx, (input_ids, target_ids) in enumerate(tqdm(train_dataloader, desc='Epoch ' + str(epoch))):
        
        input_ids = input_ids.to(device)
        target_ids = target_ids.to(device)
        
        optimizer.zero_grad()
        
        logits, targets = model(source_tokens=input_ids, target_tokens=target_ids)
        
        loss = criterion(logits.view(-1, logits.shape[-1]), 
                        targets.contiguous().view(-1))
        
        train_loss += loss.item()
        
        loss.backward()
        
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f"Epoch {epoch}, batch {batch_idx}, loss {loss.item()}")
        if scheduler:
            scheduler.step()
            
    avg_train_loss = train_loss / len(train_dataloader)
    
    print(f"Epoch {epoch}: average loss = {avg_train_loss}")

In [11]:
def main():

    for lr in [1e-4, 5e-4, 7e-4, 1e-5, 3e-5, 5e-5, 7e-5, 9e-5]:
        print("Start Training")
    
        train_dataset = GAIIC_Dataset('./data/train_df.csv', 150, 150)
        val_dataset = GAIIC_Dataset('./data/test_df.csv', 150, 150)
        test_dataset = GAIIC_Dataset('./data/preliminary_a_test.csv', 150, 150)

        train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_dataloader  = DataLoader(val_dataset, batch_size=32, shuffle=False)
        test_dataloader  = DataLoader(test_dataset, batch_size=32, shuffle=False)
        model = T5_Model().to('cuda')

        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]

        optimizer_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.01,
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.0,
            },
        ]
        optimizer = optim.AdamW(optimizer_parameters, lr=lr,  weight_decay=0.01, eps=1e-8)

        criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

        epochs = 7
        bst_val_cider = 0
        for epoch in range(epochs):
            train_fn(model, train_dataloader, criterion, optimizer, scheduler=None, epoch=epoch)

            cider_score, cider_scores = val_fn(model, val_dataloader)

            if cider_score >= bst_val_cider:
                best_model_wts = copy.deepcopy(model.state_dict())
                bst_val_cider = cider_score

        print(f"Best val cider: {bst_val_cider}")
        model.load_state_dict(best_model_wts)

        test_outputs = test(model, test_dataloader)

        df = pd.DataFrame(test_outputs)
        df['caption'] = df['caption'].apply(lambda x: x[0])
        df.to_csv(f'./data_sub_lr_{lr}_cider_{bst_val_cider}.csv', header=None, index=False)
        
        torch.cuda.empty_cache()

In [12]:
test_outputs = main()

Start Training


Epoch 0:   0%|          | 1/282 [00:01<05:22,  1.15s/it]

Epoch 0, batch 0, loss 7.701681613922119


Epoch 0:  36%|███▌      | 101/282 [01:27<02:37,  1.15it/s]

Epoch 0, batch 100, loss 2.6557552814483643


...

Epoch 6: 100%|██████████| 282/282 [04:03<00:00,  1.16it/s]


Epoch 6: average loss = 1.412235528019303


100%|██████████| 63/63 [00:45<00:00,  1.38it/s]


cider_score: 0.6723483030455616 [1.24486624e-03 5.29553752e-01 4.86108283e-02 ... 6.12299171e-01
 4.68069768e-01 2.63103707e-04]
Best val cider: 0.9132030191528517


100%|██████████| 94/94 [01:05<00:00,  1.44it/s]
